This notebook gives an example of a multi seasonal AR model in orbit.
There are three parts: Data is simulated, Orbit ARMA and statsmodel ARMA are fit, and results are plotted / compared. 

# Preliminary Test on Orbit ARMA Model

In [ ]:
from datetime import datetime, timedelta
import datetime as dt 

import pandas as pd
import numpy as np
import torch

import matplotlib.pyplot as plt
%matplotlib inline

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_predict

import orbit
from orbit.models import DLT,ETS, KTRLite, LGT, ARMA
from orbit.utils.simulation import make_trend, make_regression
from orbit.diagnostics.plot import plot_predicted_data, plot_predicted_components
from orbit.diagnostics.metrics import smape

In [ ]:
%load_ext autoreload
%autoreload 2

## Data Simulation
AR 1,24

In [ ]:
# make the data
n = 744 # hourly for 1 month 


eta = np.random.normal(0, 1.0, n)

rho = [0.60, -0.15]
y = eta.copy()
for i in range(2, n):
    y[i] = rho[0]*y[i-1] + rho[1]*y[i-24] +eta[i]
     

plt.plot(y)


In [ ]:
def daterange(start_date, end_date):
    delta = timedelta(hours=1)
    while start_date < end_date:
        yield start_date
        start_date += delta
        
hours = [datetime(2013, 1, 1, 14, 00)]        

start_date = datetime(2021, 1, 1, 00, 00)
end_date = datetime(2022, 1, 1, 00, 00)
i = 0
for single_date in daterange(start_date, end_date):
    hours[i]=hours.append(single_date.strftime("%Y-%m-%d %H:%M"))       
    i += i
hours = hours[1:(n+1)]
hours = [dt.datetime.strptime(hour, "%Y-%m-%d %H:%M") for hour in hours]

In [ ]:
df = pd.DataFrame()
df['hours'] = hours
df['y'] = y

In [ ]:
df_train = df[0:372]
df_test = df[372:n]

## Prediction Comparison orbit-ml vs. statsmodels

### AR 1, 24

In [ ]:
# orbit model and prediction 
mod = ARMA(
        level_first=0,
        estimator='stan-mcmc',
        date_col='hours',
        response_col='y',
        #ma_lags = [1] ,
        ar_lags = [1,24],
    num_warmup=10000,
    num_sample=50000,
)
mod.fit(df_train)
pred = mod.predict(df)
pred = pred.rename(columns={
    'prediction': 'prediction_orbit',
    'prediction_5': 'prediction_5_orbit',
    'prediction_95': 'prediction_95_orbit',
})

In [ ]:
# stats model and prediction 
y = pd.Series(df_train['y'])
arma_mod = ARIMA(y, order=(1, 0, 0), seasonal_order= (1,0,0,24))
arma_res = arma_mod.fit()
forecast = arma_res.get_forecast(372)
yhat = forecast.predicted_mean
yhat_conf_int = forecast.conf_int(alpha=0.10)
pred['prediction_5_statsmodel'] = yhat_conf_int['lower y']
pred['prediction_statsmodel'] = yhat
pred['prediction_95_statsmodel'] = yhat_conf_int['upper y']

# Compare Orbit and Statsmodel

In [ ]:
#plot the two predictions 
fig, ax = plt.subplots(1, 1, figsize=(16, 8))
#plt.xlim(['2021-01-16 12:00:00', '2021-01-31 12:00:00'])
ax.set_xlim([dt.datetime(2021, 1, 16, 12,0,0), dt.datetime(2021, 1, 31, 0,0,0)])
ax.plot(pred['hours'], pred['prediction_orbit'], label='orbit', color = "blue")
ax.plot(pred['hours'], pred['prediction_statsmodel'], label='statsmodel', color = "red")
ax.plot(pred['hours'], pred['prediction_5_orbit'], label='orbit 5',  color = "blue")
ax.plot(pred['hours'], pred['prediction_5_statsmodel'], label='statsmodel 5', color = "red")
ax.plot(pred['hours'], pred['prediction_95_orbit'], label='orbit 95', color = "blue")
ax.plot(pred['hours'], pred['prediction_95_statsmodel'], label='statsmodel 95', color = "red")
ax.legend()


fig, ax = plt.subplots(1, 1, figsize=(16, 8))
#plt.xlim(['2021-01-16 12:00:00', '2021-01-31 12:00:00'])
ax.set_xlim([dt.datetime(2021, 1, 16, 12,0,0), dt.datetime(2021, 1, 18, 0,0,0)])
ax.plot(pred['hours'], pred['prediction_orbit'], label='orbit', color = "blue")
ax.plot(pred['hours'], pred['prediction_statsmodel'], label='statsmodel', color = "red")
ax.plot(pred['hours'], pred['prediction_5_orbit'], label='orbit 5',  color = "blue")
ax.plot(pred['hours'], pred['prediction_5_statsmodel'], label='statsmodel 5', color = "red")
ax.plot(pred['hours'], pred['prediction_95_orbit'], label='orbit 95', color = "blue")
ax.plot(pred['hours'], pred['prediction_95_statsmodel'], label='statsmodel 95', color = "red")
ax.legend()


In [ ]:
# orbit 
A = mod.get_posterior_samples()
print('mu'+' '+ str(np.percentile(A['mu'], (2.5, 50, 97.5)) ))
print('rho_1'+' '+str(np.percentile(A['rho'][:,0], (2.5, 50, 97.5))))
print('rho_24'+' '+str(np.percentile(A['rho'][:,1], (2.5, 50, 97.5))))
print('sigma'+' '+str(np.percentile(A['obs_sigma'], (2.5, 50, 97.5))))

In [ ]:
# statsmodel
arma_res.summary()